### Managing State


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

#### Input & Output State

In [2]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

model = ChatOpenAI()

In [3]:
from typing import TypedDict


class ChatMessages(TypedDict):
    question: str
    answer: str
    llm_calls: int

In [4]:
def call_model(state: ChatMessages):
    question = state["question"]
    llm_calls = state.get("llm_calls", 0)
    state["llm_calls"] = llm_calls + 1
    print("LLM_CALLS:", state["llm_calls"])
    response = model.invoke(input=question)
    state["answer"] = response.content
    return state

In [5]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [7]:
graph.invoke(input={"question": "Where is Silicon Valley?"})

LLM_CALLS: 1


{'question': 'Where is Silicon Valley?',
 'answer': 'Silicon Valley is located in the southern part of the San Francisco Bay Area in Northern California, United States.',
 'llm_calls': 1}

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

class InputState(TypedDict):
    question: str

class PrivateState(TypedDict):
    llm_calls: int

class OutputState(TypedDict):
    answer: str

class OverallState(InputState, PrivateState, OutputState):
    pass

In [10]:
workflow = StateGraph(state_schema=OverallState, input_schema=InputState, output_schema=OutputState)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [12]:
graph.invoke(input={"question": "Where is Silicon Valley?"})

LLM_CALLS: 1


{'answer': 'Silicon Valley is located in northern California, in the southern part of the San Francisco Bay Area. It encompasses a number of cities and communities, including Palo Alto, Mountain View, Cupertino, and San Jose.'}

#### Run Time Configuration

In [13]:
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.config import RunnableConfig
from langchain.schema import SystemMessage, HumanMessage


def call_model(state: OverallState, config: RunnableConfig):
    language = config["configurable"].get("language", "English")
    system_message_content = "Respond in {}".format(language)
    system_message = SystemMessage(content=system_message_content)
    messages = [system_message, HumanMessage(content=state["question"])]
    response = model.invoke(messages)
    return {"answer": response}


In [14]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [15]:
config = {"configurable": {"language": "hindi"}}
graph.invoke(input={"question": "Where is Silicon Valley?"}, config=config)

{'question': 'Where is Silicon Valley?',
 'answer': AIMessage(content='सिलिकॉन वैली कैलिफोर्निया, संयुक्त राज्य अमेरिका में स्थित है। यह सान फ्रांसिस्को उपनगरीय क्षेत्र में है और तकनीकी कंपनियों के मुख्य केंद्र के रूप में मशहूर है।', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 19, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C5ESPzbou3azieaVmyK8bN7cjF8sr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--380fe55c-e03e-4d8f-8750-7123e6ef1082-0', usage_metadata={'input_tokens': 19, 'output_tokens': 156, 'total_tokens': 175, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [16]:
config = {"configurable": {"language": "german"}}
graph.invoke(input={"question": "Where is Silicon Valley?"}, config=config)

{'question': 'Where is Silicon Valley?',
 'answer': AIMessage(content='Silicon Valley befindet sich im Bundesstaat Kalifornien in den Vereinigten Staaten. Es ist in der Nähe von San Francisco und erstreckt sich über mehrere Städte wie Palo Alto, Mountain View und Cupertino. Es ist bekannt für seine hohe Konzentration an Technologieunternehmen und Start-ups.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 19, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C5ESWnNYQTPS6VVHC0mUY8gr0LbA3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--86c612a5-57ff-4a9a-bada-4e4405376695-0', usage_metadata={'input_tokens': 19, 'output_tokens': 70, 'tot